In [312]:
import collections
import http.client
import pandas as pd
from operator import itemgetter
import json
from collections import namedtuple
pd.set_option("display.max_rows", 200)
pd.set_option('display.max_colwidth', -1)

In [313]:
Rounds = namedtuple('Rounds', 'player round score strokes thru')

In [314]:
conn = http.client.HTTPConnection("api.sportradar.us")
conn.request("GET", "/golf-t2/leaderboard/pga/2018/tournaments/b404a8d5-5e33-4417-ae20-5d4d147042ee/leaderboard.json?api_key=u2rg6jhkezj6dkutjcmfqc8e")
res = conn.getresponse()
data = res.read()
d = data.decode("utf-8")

In [315]:
missed_cut = ["Sandy Lyle", "Charl Schwartzel", "Jose Maria Olazabal", "Jason Dufner", "Patrick Cantlay", 
              "Shubhankar Sharma", "Thomas Pieters", "Ted Potter", "Brendan Steele", "Dylan Frittelli", 
              "Danny Willett", "Ross Fisher", "Wesley Bryan", "Pat Perez", "Patton Kizzire", "Trevor Immelman", 
              "Joaquin Niemann", "Yuta Ikeda", "Alexander Noren", "Kevin Chappell", "Doc Redman", "Gary Woodland", 
              "Yusaku Miyazato", "Austin Cook", "Billy Horschel", "Ian Woosnam", "Mike Weir", "Larry Mize", 
              "Angel Cabrera", "Yuxin Lin", "Mark O'Meara", "Matt Parziale", "Sergio Garcia", "Harry Ellis"]
missed_cut_players = [p.lower().replace(" ", "") for p in missed_cut]
sorted(list(lb_df[lb_df["status"] == 'CUT'].full_name)) == sorted(missed_cut_players)

True

In [316]:
cols = ["first_name", "last_name", "position", "score", "strokes", "full_name", "rounds"]
lb_df = pd.DataFrame(json.loads(d)['leaderboard'])
lb_df["full_name"] = lb_df["first_name"].str.lower().str.replace(" ", "") + lb_df["last_name"].str.lower().str.replace(" ", "")
lb_df[cols]
lb_df.index += 1
lb_df.to_csv("leaderboard.csv", columns=cols)
lb_df[[c for c in cols if c not in ("rounds",)]]

,first_name,last_name,position,score,strokes,full_name
1,Patrick,Reed,1,-15,265,patrickreed
2,Rickie,Fowler,2,-13,271,rickiefowler
3,Jordan,Spieth,2,-13,275,jordanspieth
4,Jon,Rahm,4,-11,273,jonrahm
5,Cameron,Smith,5,-9,279,cameronsmith
6,Rory,McIlroy,5,-9,271,rorymcilroy
7,Bubba,Watson,5,-9,279,bubbawatson
8,Henrik,Stenson,8,-8,276,henrikstenson
9,Marc,Leishman,8,-8,280,marcleishman
10,Tony,Finau,10,-7,281,tonyfinau


In [317]:
leader = lb_df.iloc[0].full_name
leader

'patrickreed'

In [318]:
players_and_rounds = [("{} {}".format(r["first_name"], r["last_name"]), 
                       r["rounds"][2], r["rounds"][3]) for r in json.loads(d)["leaderboard"] if len(r["rounds"]) > 2]
round3 = [Rounds(p[0], p[1]["sequence"], p[1]["score"], p[1]["strokes"], p[1]["thru"]) for p in players_and_rounds]
round4 = [Rounds(p[0], p[2]["sequence"], p[2]["score"], p[2]["strokes"], p[2]["thru"]) for p in players_and_rounds]
round4_df = pd.DataFrame(sorted(round4, key=lambda x: x.thru))
round4_df.index += 1
round4_df.to_csv("round4_leaderboard.csv")
round4_df

,player,round,score,strokes,thru
1,Patrick Reed,4,-1,63,16
2,Rory McIlroy,4,2,66,16
3,Rickie Fowler,4,-4,64,17
4,Jon Rahm,4,-3,65,17
5,Henrik Stenson,4,-1,67,17
6,Tommy Fleetwood,4,1,69,17
7,Jordan Spieth,4,-8,64,18
8,Cameron Smith,4,-6,66,18
9,Bubba Watson,4,-3,69,18
10,Marc Leishman,4,-2,70,18


In [319]:
LEADER = 3
def get_high_round(rounds):
    high_player = max(rounds, key=lambda x: x.strokes)
    return high_player
    
def get_score(player):
    player = player.lower().replace(" ", "")
    score = lb_df.loc[lb_df["full_name"] == player]["score"]
    strokes = lb_df.loc[lb_df["full_name"] == player]["strokes"]
    high_round3 = get_high_round(round3)
    high_round4 = get_high_round(round4)
    high_scores = high_round3.score + high_round4.score
    high_strokes = high_round3.strokes + high_round4.strokes
    if player in missed_cut_players:
        return "**MC** Score: {}, Strokes: {}".format(int(score) + high_scores, int(strokes) + high_strokes)
    if player == leader:
        return "**LEADER** Score: {}, Strokes: {}".format(int(score) - LEADER, int(strokes) - LEADER)
    return "Score: {}, Strokes: {}".format(int(score), int(strokes))
    
def get_strokes(player):
    player = player.lower().replace(" ", "")
    high_strokes = 0
    if player in missed_cut_players:
        high_strokes = get_high_round(round3).strokes + get_high_round(round4).strokes
        return int(lb_df.loc[lb_df["full_name"] == player]["strokes"]) + high_strokes
    if player == leader:
        return int(lb_df.loc[lb_df["full_name"] == player]["strokes"]) - LEADER
    return int(lb_df.loc[lb_df["full_name"] == player]["strokes"])

In [320]:
df = pd.read_csv("masters.csv")
df = df[["Pool Player", "Player1", "Player2", "Player3", "Player4", 
         "Player5", "Player6", "Player7", "Total", "Tie Breaker"]]

In [321]:
for index, row in df.iterrows():
    score_accum = 0
    for player in row.values[1:8]:
        score_accum += get_strokes(player)
    df.set_value(index, "Total", score_accum)

In [322]:
df = df.sort_values(by=['Total']).reset_index(drop=True)
for i in range(1,8):
    df["Score{}".format(i)] = df["Player{}".format(i)].apply(lambda x: get_score(x))

In [323]:
high_round3 = get_high_round(round3)
high_round4 = get_high_round(round4)
for r in (high_round3, high_round4,):  
    print("High round {} from {}, with score: {}, strokes: {}, thru: {}".format(r.round, r.player, r.score, r.strokes, r.thru))

High round 3 from Vijay Singh, with score: 7, strokes: 79, thru: 18
High round 4 from Kyle Stanley, with score: 4, strokes: 76, thru: 18


In [324]:
df.index += 1
df = df[["Pool Player", 
         "Player1", "Score1",
         "Player2", "Score2",
         "Player3", "Score3",
         "Player4", "Score4",
         "Player5", "Score5",
         "Player6", "Score6",
         "Player7", "Score7",
         "Total", "Tie Breaker"]]
df

,Pool Player,Player1,Score1,Player2,Score2,Player3,Score3,Player4,Score4,Player5,Score5,Player6,Score6,Player7,Score7,Total,Tie Breaker
1,Sam Montgomery,Jordan Spieth,"Score: -13, Strokes: 275",Tommy Fleetwood,"Score: -5, Strokes: 279",Charley Hoffman,"Score: -6, Strokes: 282",Branden Grace,"Score: -1, Strokes: 287",Adam Hadwin,"Score: -1, Strokes: 287",Jimmy Walker,"Score: -2, Strokes: 286",Bryson DeChambeau,"Score: 3, Strokes: 291",1987.0,282.0
2,Dave Peterson,Rory McIlroy,"Score: -9, Strokes: 271",Jason Day,"Score: -2, Strokes: 286",Patrick Reed,"**LEADER** Score: -18, Strokes: 262",Tony Finau,"Score: -7, Strokes: 281",Jason Dufner,"**MC** Score: 17, Strokes: 305",Tiger Woods,"Score: 1, Strokes: 289",Martin Kaymer,"Score: 6, Strokes: 294",1988.0,278.0
3,Jeremy Szyba #3,Dustin Johnson,"Score: -7, Strokes: 281",Phil Mickelson,"Score: 2, Strokes: 290",Patrick Reed,"**LEADER** Score: -18, Strokes: 262",Branden Grace,"Score: -1, Strokes: 287",Adam Hadwin,"Score: -1, Strokes: 287",Daniel Berger,"Score: 1, Strokes: 289",Ian Poulter,"Score: 4, Strokes: 292",1988.0,278.0
4,Chris Stine,Rory McIlroy,"Score: -9, Strokes: 271",Phil Mickelson,"Score: 2, Strokes: 290",Brian Harman,"Score: 4, Strokes: 292",Louis Oosthuizen,"Score: -6, Strokes: 282",Cameron Smith,"Score: -9, Strokes: 279",Tiger Woods,"Score: 1, Strokes: 289",Ryan Moore,"Score: 0, Strokes: 288",1991.0,280.0
5,Zachary Sight,Justin Thomas,"Score: -4, Strokes: 284",Phil Mickelson,"Score: 2, Strokes: 290",Patrick Reed,"**LEADER** Score: -18, Strokes: 262",Daniel Berger,"Score: 1, Strokes: 289",Siwoo Kim,"Score: -1, Strokes: 287",Tiger Woods,"Score: 1, Strokes: 289",Fred Couples,"Score: 3, Strokes: 291",1992.0,278.0
6,Joe Kearney,Justin Thomas,"Score: -4, Strokes: 284",Jason Day,"Score: -2, Strokes: 286",Patrick Reed,"**LEADER** Score: -18, Strokes: 262",Louis Oosthuizen,"Score: -6, Strokes: 282",Thomas Pieters,"**MC** Score: 18, Strokes: 306",Tiger Woods,"Score: 1, Strokes: 289",Russell Henley,"Score: -5, Strokes: 283",1992.0,279.0
7,Tristen Woods,Jon Rahm,"Score: -11, Strokes: 273",Henrik Stenson,"Score: -8, Strokes: 276",Patrick Reed,"**LEADER** Score: -18, Strokes: 262",Brendan Steele,"**MC** Score: 18, Strokes: 306",Cameron Smith,"Score: -9, Strokes: 279",Billy Horschel,"**MC** Score: 22, Strokes: 310",Adam Scott,"Score: 1, Strokes: 289",1995.0,279.0
8,Jeremy Szyba,Rickie Fowler,"Score: -13, Strokes: 271",Paul Casey,"Score: -5, Strokes: 283",Bubba Watson,"Score: -9, Strokes: 279",Louis Oosthuizen,"Score: -6, Strokes: 282",Jason Dufner,"**MC** Score: 17, Strokes: 305",Jimmy Walker,"Score: -2, Strokes: 286",Zach Johnson,"Score: 2, Strokes: 290",1996.0,277.0
9,Paul Bower,Dustin Johnson,"Score: -7, Strokes: 281",Jason Day,"Score: -2, Strokes: 286",Bubba Watson,"Score: -9, Strokes: 279",Daniel Berger,"Score: 1, Strokes: 289",Adam Hadwin,"Score: -1, Strokes: 287",Tiger Woods,"Score: 1, Strokes: 289",Jimmy Walker,"Score: -2, Strokes: 286",1997.0,268.0
10,Kyle Ritchie,Rory McIlroy,"Score: -9, Strokes: 271",Phil Mickelson,"Score: 2, Strokes: 290",Bubba Watson,"Score: -9, Strokes: 279",Louis Oosthuizen,"Score: -6, Strokes: 282",Jason Dufner,"**MC** Score: 17, Strokes: 305",Tiger Woods,"Score: 1, Strokes: 289",Russell Henley,"Score: -5, Strokes: 283",1999.0,277.0


In [325]:
df.to_csv("masters_results.csv")

# Tabula Fun

In [102]:
# from tabula import read_pdf

In [103]:
# df = read_pdf("/Users/jeremyszyba/Desktop/2018 Masters.pdf")
# df = df.drop(df.index[10])
# df